<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化器（オプティマイザ）

機械学習を行うときに **初期学習率 (learning rate)** を設定するのが重要であることは、すでに理解されているものと思います。これによって学習の速度が変わり、大きすぎると過学習が起きるし、小さすぎると学習の進みが遅くなりますね。

ただ、学習が進むにつれて学習率は徐々に調節すべきです。最初の段階では、モデルがどのようにデータを学べばよいのか全く分からないため、大きな学習率で大まかな方向性を探ります。しかし、学習が進むにつれて、モデルが正しい方向に近づいていきます。この段階では、学習率を小さくして慎重に調整する必要があります。大きな学習率のままだと、良い解に近づいているのに、再び遠ざかってしまう可能性があります。

### そもそも最小化とは

機械学習の目的は、モデルが誤差（損失関数）を最小限に抑えることです。損失関数とは、モデルがどれだけ正しい予測をしているかを数値で示したものです。例えば、予測と実際の値が大きく離れているほど損失関数の値は大きくなり、逆に一致しているほど小さくなります。

この損失関数を最小化するためには、数学的には関数の最小値を求める「最適化問題」を解く必要があります。ここで着目すべきなのが **勾配（関数の傾き）**です。

### 最小化の手順

基本的な **勾配降下法 (Gradient Descent)** という方法で、関数が最小となるパラメータを探してみましょう。

- 勾配の計算

損失関数を入力パラメータについて微分して、**現時点からどの方向に進めば関数の値が減るか** を調べます。

- パラメータの更新

勾配が示す方向とは逆の方向に少しだけ進むことで、損失を少しずつ減らします。この際、更新の幅を決めるのが **学習率（learning rate） ** です。

- 繰り返し

この操作を何度も繰り返して、損失が最小になるパラメータを探します。

In [ ]:
# 初期値と学習率
x = 0  # 初期点
learning_rate = 0.01

# 最大ステップ数
steps = 20


import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

# 損失関数とその勾配を定義
def loss_function(x):
    return x**4 - 3*x**3 - 7*x**2 + 5*x  # 損失関数

def gradient(x):
    return 4*x**3 - 9*x**2 - 8*x + 5  # 損失関数の微分

# 記録用リスト
path = [x]

# 最適化
for _ in trange(steps):
    grad = gradient(x)  # 勾配の計算
    x -= learning_rate * grad  # パラメータの更新
    path.append(x)

# 可視化
x_vals = np.linspace(-3, 5, 500)
y_vals = loss_function(x_vals)

plt.figure(figsize=(8, 6))
plt.plot(x_vals, y_vals, label="Loss Function: $y = x^4 - 3x^3 - 4x^2 + 5x$", color="blue")
plt.scatter(path, [loss_function(x) for x in path], color="red", label="SGD Steps")
plt.title("Gradient Descent on $y = x^4 - 3x^3 - 4x^2 + 5x$")
plt.xlabel("x")
plt.ylabel("y")
plt.axhline(0, color="black", linestyle="--", linewidth=0.5)
plt.axvline(0, color="black", linestyle="--", linewidth=0.5)
plt.legend()
plt.grid()
plt.show()


## 演習１

初期値や学習率を調節して、大域解に到達するようにしてください。

また、なるべく高速に（少ない回数で）到達できるようにしてください。初期値が適切でも、あまりにも学習率を上げすぎるとうまくいかないことも確認してください。

----

### 確率的勾配降下法（SGD）

最小化を効率よく行うために用いられるのが **確率的勾配降下法（Stochastic Gradient Descent, SGD）** です。上記の勾配降下法に加えてミニバッチを使うため、そのため、毎回の最適化には不確実性があります。

### 演習２

SGD を用いて iris の分類問題を実行し、どのようなスピードで学習率が低下していくのか確認してみましょう。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# Iris データセットの読み込み
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Virginica とそれ以外の2クラスに限定
y = (y == 2).astype(int)  # Virginica: 1, それ以外: 0

# 特徴量を2次元に限定（Sepal Length, Sepal Width）
X = X[:, :2]

# データの標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# サンプル点の可視化
plt.figure(figsize=(8, 6))
plt.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], c="green", marker="o", label="Train - Not Versicolor")
plt.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], c="blue", marker="o", label="Train - Versicolor")
plt.scatter(X_test[y_test == 0, 0], X_test[y_test == 0, 1], c="green", marker="x", label="Test - Not Versicolor")
plt.scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], c="blue", marker="x", label="Test - Versicolor")
plt.title("Train/Test Split Visualization (Versicolor vs Others)")
plt.xlabel("Feature 1 (Standardized)")
plt.ylabel("Feature 2 (Standardized)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
from logging import critical
# オプティマイザのリスト
optimizers = {
    "SGD": lambda: optim.SGD(model.parameters(), lr=0.1),
    # 他にも 「"名前": lambda: 呼び出し文」 という書き方で追加してください
}


# シンプルなMLPモデル
model = nn.Sequential(
    nn.Linear(2, 3),  # 特徴量数: 2, 出力: 3（3クラス分類）
    nn.Softmax(dim=1)  # 出力を確率分布に変換
)

criterion = nn.CrossEntropyLoss()  # クロスエントロピー損失を定義

In [ ]:
# トレーニング関数
def train(optimizer_name):
    optimizer = optimizers[optimizer_name]()  # オプティマイザを作成
    scheduler = StepLR(optimizer, step_size=50, gamma=0.9)  # 学習率スケジューラ
    losses = []  # ロスを記録するリスト
    lrs = []     # 学習率を記録するリスト

    for epoch in range(300):
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step()  # 学習率を更新

        # 各エポックでのロスと学習率を記録
        losses.append(loss.item())
        lrs.append(optimizer.param_groups[0]['lr'])

    return losses, lrs  # ロスと学習率のリストを返す

# トレーニングと可視化
for optimizer_name in optimizers.keys():
    model.apply(lambda m: m.reset_parameters() if hasattr(m, "reset_parameters") else None)  # 重み初期化
    losses, lrs = train(optimizer_name)  # 修正後の train 関数

    # 横並びのグラフ作成
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # 学習曲線と学習率の変化を左側にプロット
    ax1 = axes[0]
    ax2 = ax1.twinx()
    ax1.plot(losses, label="Loss", color="blue")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color="blue")
    ax1.tick_params(axis='y', labelcolor="blue")
    ax2.plot(lrs, label="Learning Rate", color="red")
    ax2.set_ylabel("Learning Rate", color="red")
    ax2.tick_params(axis='y', labelcolor="red")
    ax1.set_title(f"Loss and Learning Rate ({optimizer_name})")

    # 決定境界を右側にプロット
    ax3 = axes[1]
    xx, yy = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
    preds = model(grid).detach().numpy().reshape(xx.shape)
    ax3.contourf(xx, yy, preds, levels=50, cmap="RdBu", alpha=0.3)
    ax3.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], c="red", marker="o", label="Not Virginica")
    ax3.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], c="blue", marker="s", label="Virginica")
    ax3.set_title(f"Decision Boundary ({optimizer_name})")
    ax3.legend()

    # レイアウト調整して表示
    fig.tight_layout()
    plt.show()


### 演習３

SGD 以外にも、いくつか有名なオプティマイザがあります。optimizers に追加して実行し、SGD との速度や結果の違いを見てみましょう。

| オプティマイザ名   | 特徴                                                                 | PyTorchの関数名                |
|--------------------|----------------------------------------------------------------------|--------------------------------|
| SGD               | 確率的勾配降下法。シンプルで計算コストが低いが、収束が遅いことがある。  | `torch.optim.SGD`             |
| Momentum          | SGD の改良版。前回の更新方向を加味して振動を抑え、安定した収束を実現。 | `torch.optim.SGD`（`momentum` オプションを使用） |
| AdaGrad           | パラメータごとに学習率を調整。学習率が小さくなりすぎることが課題。      | `torch.optim.Adagrad`         |
| RMSprop           | AdaGrad の学習率低下問題を解決。適応的な学習率を維持し、安定した学習。  | `torch.optim.RMSprop`         |
| Adam              | Momentum と RMSprop を組み合わせたもの。高速収束と安定性が特徴。       | `torch.optim.Adam`            |
